In [1]:
import numpy as np
import random
import sklearn_crfsuite
import nltk
import gensim

In [2]:
with open('ner.txt') as f:
    content = f.readlines()

data = []
data1 = []
sent = []
num_sent = 0;
label=[]
label_sent = []
for content_data in content:
    if not content_data == '\n':
        d1 = content_data.strip()
        d2 = d1.split()
        sent.append(d2[0])
        label_sent.append(d2[1])
    else:
        data1.append(sent)
        sent = nltk.pos_tag(sent)
        data.append(sent)
        label.append(label_sent)
        sent = []
        label_sent = []
        num_sent = num_sent + 1



In [3]:
model = gensim.models.Word2Vec(data1,min_count=1,size = 50)


In [4]:
K = 3
from sklearn.cluster import KMeans
embeddings = model[model.wv.vocab]
kmeans = KMeans(n_clusters=K)
kmeans.fit(embeddings)


assigned_cluster = []
for i in range(len(data1)):
    sent = model[data1[i]]
    assigned_cluster.append(kmeans.predict(sent))

/home/chaikesh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/chaikesh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [5]:
def word2features(sent, cluster_sent,i):
    word = sent[i][0]
    postag = sent[i][1]
    cluster = cluster_sent[i]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'word2VecCluster': cluster,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

In [6]:
def sent2features(sent,cluster_data):
    return [word2features(sent,cluster_data, i) for i in range(len(sent))]




num_train = int(np.floor(num_sent*0.7))
num_valid = int(np.floor(num_sent*0.1))
num_test = num_sent - num_train - num_valid

idx = list(range(num_sent))
random.shuffle(idx)

train_data =[]
train_label =[]
valid_data = []
valid_label = []
test_data = []
test_label =[]
train_cluster = []
valid_cluster= []
test_cluster = []

for i in range(0,num_train):
    train_data.append(data[idx[i]])
    train_label.append(label[idx[i]])
    train_cluster.append(assigned_cluster[idx[i]])

for i in range(num_train,num_train+num_valid):
    valid_data.append(data[idx[i]])
    valid_label.append(label[idx[i]])
    valid_cluster.append(assigned_cluster[idx[i]])

for i in range(num_train+num_valid,num_train+num_valid+num_test):
    test_data.append(data[idx[i]])
    test_label.append(label[idx[i]])
    test_cluster.append(assigned_cluster[idx[i]])

In [13]:
X_train = []
for i in range(len(train_data)):
    X_train.append(sent2features(train_data[i],train_cluster[i]))

Y_train = train_label



X_test = []
for i in range(len(test_data)):
    X_test.append(sent2features(test_data[i],test_cluster[i]))

Y_test = test_label

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, Y_train)
Y_pred = crf.predict(X_test)

In [14]:

from sklearn.metrics import classification_report



dicts = {"O":0,"D":1,"T":2}

Y_test_dicts = []
Y_pred_dicts = []
for i in range(len(test_data)):
    Y_test_dicts  =  Y_test_dicts + ([dicts[s] for s in Y_test[i]])
    Y_pred_dicts  = Y_pred_dicts + ([dicts[s] for s in Y_pred[i]])

Y_test_dicts = np.array(Y_test_dicts)
Y_pred_dicts = np.array(Y_pred_dicts)

match_score = classification_report(Y_test_dicts,Y_pred_dicts)
print match_score

             precision    recall  f1-score   support

          0       0.95      0.98      0.96     11392
          1       0.81      0.69      0.74      1049
          2       0.77      0.56      0.65       747

avg / total       0.92      0.93      0.93     13188



In [11]:
match_score

'             precision    recall  f1-score   support\n\n          0       0.95      0.98      0.96     11392\n          1       0.81      0.69      0.74      1049\n          2       0.77      0.56      0.65       747\n\navg / total       0.92      0.93      0.93     13188\n'